In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
from keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout, Input, Concatenate, Conv2D, BatchNormalization
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Dropout, LSTM, GRU
from tensorflow.keras.utils import to_categorical
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
def load_ravdess_data():
    emotion_files = {'neutral': '/kaggle/input/ravdess-features/Neutral.xlsx',
                     'calm': '/kaggle/input/ravdess-features/Calm.xlsx',
                     'happy': '/kaggle/input/ravdess-features/Happy.xlsx',
                     'sad': '/kaggle/input/ravdess-features/Sad.xlsx',
                     'angry': '/kaggle/input/ravdess-features/Angry.xlsx',
                     'fearful': '/kaggle/input/ravdess-features/Fearful.xlsx',
                     'disgust': '/kaggle/input/ravdess-features/Disgusted.xlsx',
                     'surprised': '/kaggle/input/ravdess-features/Surprised.xlsx'}
    
    all_data = []
    
    for emotion, file_path in emotion_files.items():
        try:
            df = pd.read_excel(file_path)
            df['emotion'] = emotion
            all_data.append(df)
            print(f"Loaded {emotion} data: {len(df)} samples")
        except Exception as e:
            print(f"Error loading {emotion} file: {str(e)}")
    
    combined_df = pd.concat(all_data, ignore_index=True)
    
    feature_columns = ['face_x', 'face_y', 'face_width', 'face_height', 'face_aspect_ratio',
                       'left_eye_x', 'left_eye_y', 'left_eye_width', 'left_eye_height',
                       'right_eye_x', 'right_eye_y', 'right_eye_width', 'right_eye_height',
                       'eye_separation', 'mouth_x', 'mouth_y', 'mouth_width', 'mouth_height',
                       'mouth_aspect_ratio', 'avg_intensity', 'intensity_variance']
    
    print("\nDataset Statistics:")
    print(f"Total samples: {len(combined_df)}")
    print("\nSamples per emotion:")
    print(combined_df['emotion'].value_counts())
    
    X = combined_df[feature_columns].values
    
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(combined_df['emotion'])
    
    return X, y, label_encoder

In [ ]:
class EmotionDetectionModels:
    def __init__(self, X, y, label_encoder):
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            X, y, test_size = 0.2, random_state = 42, stratify = y)
        
        self.scaler = StandardScaler()
        self.X_train_scaled = self.scaler.fit_transform(self.X_train)
        self.X_test_scaled = self.scaler.transform(self.X_test)
        
        self.y_train_cat = to_categorical(self.y_train)
        self.y_test_cat = to_categorical(self.y_test)
        self.label_encoder = label_encoder
        
        self.results = {}
        print(self.X_train)
        print(self.X_train_scaled)
        print(f"\nTraining set shape: {self.X_train.shape}")
        print(f"Testing set shape: {self.X_test.shape}")
        
    def create_model(self):
        inputs = Input(shape = (self.X_train.shape[1],))
        
        x = Dense(256, activation = 'relu')(inputs)
        x = BatchNormalization()(x)
        x = Dropout(0.3)(x)
        
        x = Dense(128, activation = 'relu')(x)
        x = BatchNormalization()(x)
        x = Dropout(0.3)(x)
        
        x = Dense(64, activation = 'relu')(x)
        x = BatchNormalization()(x)
        x = Dropout(0.3)(x)
        
        outputs = Dense(8, activation = 'softmax')(x)
        
        model = Model(inputs = inputs, outputs = outputs)
        model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
        model.summary()
        return model
    
    def create_lstm(self):
        X_train_reshaped = self.X_train_scaled.reshape((self.X_train_scaled.shape[0], 1, self.X_train_scaled.shape[1]))
        X_test_reshaped = self.X_test_scaled.reshape((self.X_test_scaled.shape[0], 1, self.X_test_scaled.shape[1]))
        
        model = Sequential([
            LSTM(128, input_shape = (1, self.X_train.shape[1])),
            Dropout(0.3),
            Dense(64, activation = 'relu'),
            Dropout(0.2),
            Dense(8, activation = 'softmax')
        ])
        model.compile(optimizer = 'adam', loss='categorical_crossentropy', metrics=['accuracy'])
        return model, X_train_reshaped, X_test_reshaped

    def create_gru(self):
        X_train_reshaped = self.X_train_scaled.reshape((self.X_train_scaled.shape[0], 1, self.X_train_scaled.shape[1]))
        X_test_reshaped = self.X_test_scaled.reshape((self.X_test_scaled.shape[0], 1, self.X_test_scaled.shape[1]))
        
        model = Sequential([
            GRU(128, input_shape=(1, self.X_train.shape[1])),
            Dropout(0.3),
            Dense(64, activation='relu'),
            Dropout(0.2),
            Dense(8, activation='softmax')
        ])
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        return model, X_train_reshaped, X_test_reshaped

    def plot_confusion_matrix(self, conf_matrix, title, emotion_labels):
        plt.figure(figsize=(10, 8))
        sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
                    xticklabels=emotion_labels, yticklabels=emotion_labels)
        plt.title(title)
        plt.ylabel('True Label')
        plt.xlabel('Predicted Label')
        plt.show()

    def plot_training_history(self, history, title):
        plt.figure(figsize=(12, 4))
        
        plt.subplot(1, 2, 1)
        plt.plot(history['accuracy'], label='Training Accuracy')
        plt.plot(history['val_accuracy'], label='Validation Accuracy')
        plt.title(f'{title} - Accuracy')
        plt.legend()
        
        plt.subplot(1, 2, 2)
        plt.plot(history['loss'], label='Training Loss')
        plt.plot(history['val_loss'], label='Validation Loss')
        plt.title(f'{title} - Loss')
        plt.legend()
        
        plt.tight_layout()
        plt.show()

    def train_and_evaluate(self, model_name, model, X_train, X_test, epochs=1000, batch_size=32):
        print(f"\nTraining {model_name}...")
        
        history = model.fit(
            X_train, self.y_train_cat,
            epochs=epochs,
            batch_size=batch_size,
            validation_split=0.2,
            verbose=1
        )
        
        y_pred = model.predict(X_test)
        y_pred_classes = np.argmax(y_pred, axis=1)
        y_test_classes = np.argmax(self.y_test_cat, axis=1)
        
        y_test_emotions = self.label_encoder.inverse_transform(y_test_classes)
        y_pred_emotions = self.label_encoder.inverse_transform(y_pred_classes)
        
        conf_matrix = confusion_matrix(y_test_emotions, y_pred_emotions)
        class_report = classification_report(y_test_emotions, y_pred_emotions)
        
        self.results[model_name] = {
            'model': model,
            'history': history.history,
            'confusion_matrix': conf_matrix,
            'classification_report': class_report,
            'predictions': y_pred_emotions,
            'true_labels': y_test_emotions
        }

        self.plot_training_history(history.history, model_name)
        self.plot_confusion_matrix(conf_matrix, f'{model_name} Confusion Matrix', 
                                 self.label_encoder.classes_)
        
        return history, conf_matrix, class_report

    def print_comparison(self):
        print("\nModel Comparison Results")
        print("=" * 50)
        
        summary_data = []
        
        for model_name, result in self.results.items():
            print(f"\n{model_name} Results")
            print("-" * 30)
            print("\nClassification Report:")
            print(result['classification_report'])
            
            val_acc = result['history']['val_accuracy'][-1]
            val_loss = result['history']['val_loss'][-1]
            
            summary_data.append({
                'Model': model_name,
                'Validation Accuracy': val_acc,
                'Validation Loss': val_loss
            })
        
        summary_df = pd.DataFrame(summary_data)
        print("\nSummary of Results:")
        print(summary_df.to_string(index=False))

In [ ]:
X, y, label_encoder = load_ravdess_data()

In [ ]:
my_model = EmotionDetectionModels(X, y, label_encoder)

In [ ]:
# model = my_model.create_model()
# my_model.train_and_evaluate("Model", model, my_model.X_train_scaled, my_model.X_test_scaled)

In [ ]:
lstm_model, X_train_reshaped, X_test_reshaped = my_model.create_lstm()
my_model.train_and_evaluate("LSTM", lstm_model, X_train_reshaped, X_test_reshaped)

In [ ]:
# gru_model, X_train_reshaped, X_test_reshaped = my_model.create_gru()
# my_model.train_and_evaluate("GRU", gru_model, X_train_reshaped, X_test_reshaped)

In [ ]:
my_model.print_comparison()